In [272]:
from keras.preprocessing import image
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import os
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint
from glob import glob
import random

random.seed(2000)
def dispImages(folder):
    fig=plt.figure(figsize=(20,10))
    for dirpath,_,filenames in os.walk(folder):
        for i in range(1):
            imgpath=os.path.abspath(os.path.join(dirpath,filenames[i]))
            img=image.load_img(imgpath,target_size=(128,128))
            x = image.img_to_array(img)
            axis=fig.add_subplot(6,12,i+1,xticks=[],yticks=[])
            axis.imshow(np.squeeze(x/255))
            
def loaddata(folder):
    print('Processing '+folder)
    X=[]
    Y=[]
    for dirpath,_,filenames in os.walk(folder):
        
        if len(filenames) > 1:
            for subpath,_,files in os.walk(dirpath):
                label=2
                if os.path.basename(subpath)=='melanoma':
                    label=0
                   
                elif os.path.basename(subpath)=='nevus':
                    label=1
                   
                 
                for f in files:
                    imgpath=os.path.abspath(os.path.join(dirpath,f))
                    img=image.load_img(imgpath,target_size=(64,64))
                    x = image.img_to_array(img)
                    X.append(x/255)
                   
                    Y.append(label)
    print(array(X).shape)                
    return array(X),Y

def ingestimages(filearray):
    random.shuffle(filearray)
    print('on rest')
   
    X=[]
    Y=[]
    for imagefile in filearray:
        folder=os.path.basename(os.path.dirname(imagefile))
        #print(folder)
        label=0
        if folder=='melanoma':
            label=1

        #elif folder=='nevus':
         #   label=1
                    
        img=image.load_img(imagefile,target_size=(64,64))
        x = image.img_to_array(img)
        X.append(x/255)

        Y.append(label)
    print(array(X).shape)                
    return array(X),Y

   

In [273]:
print('Ingesting images')

trainPath = 'data/train'
validPath = 'data/valid'
testPath  = 'data/test'
train_files = np.array(glob("data/train/**/*.*"))
valid_files = np.array(glob("data/valid/**/*.*"))
test_files = np.array(glob("data/test/**/*.*"))

(x_train,y_train_orig)=ingestimages(train_files)
(x_valid,y_valid_orig)=ingestimages(valid_files)
(x_test,y_test_orig)=ingestimages(test_files)

num_classes=len(np.unique(y_train_orig))



Ingesting images
on rest
(2000, 64, 64, 3)
on rest
(150, 64, 64, 3)
on rest
(600, 64, 64, 3)


In [274]:


y_train = keras.utils.to_categorical(y_train_orig,num_classes)
y_test  = keras.utils.to_categorical(y_test_orig,num_classes)
y_valid = keras.utils.to_categorical(y_valid_orig,num_classes)

print(y_train[:5])
print(y_test[:5])
print(y_valid[:5])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]
[[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [275]:
model=Sequential()
dropoutrate=0.3
kernelSize=2
model.add(Conv2D(filters=16,kernel_size=kernelSize,padding='same',activation='relu',input_shape=(64,64,3)))

model.add(Conv2D(filters=16,kernel_size=kernelSize,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(dropoutrate))


model.add(Conv2D(filters=32,kernel_size=kernelSize,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(dropoutrate))


model.add(Conv2D(filters=64,kernel_size=kernelSize,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(dropoutrate))


model.add(Conv2D(filters=128,kernel_size=kernelSize,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(dropoutrate))

model.add(Conv2D(filters=512,kernel_size=kernelSize,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))


model.add(Flatten())


model.add(Dense(2048,activation='relu'))

model.add(Dense(2,activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_667 (Conv2D)          (None, 64, 64, 16)        208       
_________________________________________________________________
conv2d_668 (Conv2D)          (None, 64, 64, 16)        1040      
_________________________________________________________________
max_pooling2d_507 (MaxPoolin (None, 32, 32, 16)        0         
_________________________________________________________________
dropout_515 (Dropout)        (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_669 (Conv2D)          (None, 32, 32, 32)        2080      
_________________________________________________________________
max_pooling2d_508 (MaxPoolin (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_516 (Dropout)        (None, 16, 16, 32)        0         
__________

In [276]:
from keras.optimizers import Adagrad, Adam


adagrad= Adagrad(lr=0.1,epsilon=None,decay=0.0)
adam= Adam(lr=0.000001)
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
checkpointer=ModelCheckpoint(filepath='best.model.weights.hdf5',verbose=1,save_best_only=True)

model.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_valid,y_valid),callbacks=[checkpointer],verbose=1,shuffle=True)

Train on 2000 samples, validate on 150 samples
Epoch 1/10
2000/2000 [==============================] - 28s 14ms/step - loss: 0.1885 - acc: 0.8075 - val_loss: 0.2000 - val_acc: 0.8000

Epoch 00001: val_loss improved from inf to 0.20000, saving model to best.model.weights.hdf5
Epoch 2/10
2000/2000 [==============================] - 19s 9ms/step - loss: 0.1870 - acc: 0.8130 - val_loss: 0.2000 - val_acc: 0.8000

Epoch 00002: val_loss did not improve from 0.20000
Epoch 3/10
2000/2000 [==============================] - 19s 9ms/step - loss: 0.1870 - acc: 0.8130 - val_loss: 0.2000 - val_acc: 0.8000

Epoch 00003: val_loss did not improve from 0.20000
Epoch 4/10
2000/2000 [==============================] - 19s 10ms/step - loss: 0.1870 - acc: 0.8130 - val_loss: 0.2000 - val_acc: 0.8000

Epoch 00004: val_loss did not improve from 0.20000
Epoch 5/10
2000/2000 [==============================] - 20s 10ms/step - loss: 0.1870 - acc: 0.8130 - val_loss: 0.2000 - val_acc: 0.8000

Epoch 00005: val_loss did

In [277]:
print('Checking model accuracy....')
model.load_weights('best.model.weights.hdf5')
score=model.evaluate(x_test,y_test,verbose=1)
print('\n', 'Test accuracy:', score[1])

Checking model accuracy....
600/600 [==============================] - 1s 2ms/step

 Test accuracy: 0.8049999992052714
